In [ ]:
#Mounting gdrive with the colab

from google.colab import drive
drive.mount("/content/gdrive")

### Unzipping the Dog vs Cat Dataset

In [ ]:
!unzip "/content/gdrive/My Drive/Cat vs Dog/dogs-vs-cats.zip" -d "/content"

### Unzipping Train Dataset

In [ ]:
!unzip "train.zip" -d "/content"

In [ ]:
#loading basic dependencies
import numpy as np #for mathematical and array manipulation
import pandas as pd #for dataframe manipulation
import matplotlib.pyplot as plt # for plotting and visualization
import cv2 #for imagepreprocessing 
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers import Conv2D,MaxPool2D,ZeroPadding2D,Convolution2D,MaxPooling2D
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import np_utils
from keras import backend as K
from keras.layers import BatchNormalization
import numpy as np
from keras.optimizers import SGD

In [ ]:
#counting the total instances
import os 
path="/content/train"
count=0
for i in os.listdir(path):
  count=count+1
print(count)

In [ ]:
#segrating dataset between cat and dog 
image_path="train"
file_name=os.listdir(image_path)
w=150  #width
h=150  #height

target=list()
path=list()
for i in file_name:
    temp_img=i.split(".")[0]
    temp_path=os.path.join(image_path, i)
    path.append(temp_path)
    target.append(temp_img)

dataset=pd.DataFrame()
dataset['image_path']=path
dataset['target']=target

In [ ]:
dataset.head(10)  #getting first five rows

In [ ]:
dataset.describe()  #getting stastitcal information

In [ ]:
#Checking class balance
target_counts=dataset['target'].value_counts()
print("Dogs:{}".format(target_counts['dog']))
print("Cats:{}".format(target_counts['cat']))

In [ ]:
#Image Visualization for cat
plt.figure(figsize=(12,12))
for i in range(4):
    plt.subplot(1,4,i+1)
    img=cv2.imread(dataset['image_path'][i])
    plt.imshow(img)
    plt.title(dataset['target'][i])
    plt.tight_layout()
plt.show()

In [ ]:
#Image Visualization for dog
plt.figure(figsize=(12,12))
for i in range(4):
    plt.subplot(1,4,i+1)
    img=cv2.imread(dataset['image_path'][12500+i])
    plt.imshow(img)
    plt.title(dataset['target'][12500+i])
    plt.tight_layout()
plt.show()

In [ ]:
# Converting to array and label
# Dog label =1 and cat label = 0
data=[]
labels=[]
for i in range(25000):
  if dataset['target'][i]=='dog':
#    try:
    img_read=plt.imread(dataset['image_path'][i])
    img_resize=cv2.resize(img_read,(120,120))
    gray = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)
    img_array=img_to_array(gray)
    data.append(img_array)
    labels.append(1)
 #   except:
 #     None

  elif dataset['target'][i]=='cat':
    try:
      img_read=plt.imread(dataset['image_path'][i])
      img_resize=cv2.resize(img_read,(120,120))
      gray = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)
      img_array=img_to_array(gray)
      data.append(img_array)
      labels.append(0)
    except:
      None


In [ ]:
#Converting both the list to numpy arrays
image_data=np.array(data)
labels=np.array(labels)

In [ ]:
#Randomizing for even learning
idx=np.arange(image_data.shape[0])
np.random.shuffle(idx)
image_data=image_data[idx]
labels=labels[idx]

In [ ]:
#Train Test split 80:20 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(image_data,labels,test_size=0.2, random_state=101)

In [ ]:
#Converting to array
y_train=np_utils.to_categorical(y_train,num_classes=2)
y_test=np_utils.to_categorical(y_test,num_classes=2)

In [ ]:
#Modeling 
model=Sequential()
model.add(Conv2D(32, (3,3), activation="relu", input_shape=(120, 120, 1)))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation="relu"))
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation="relu"))
model.add(Conv2D(128, (3,3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation="relu"))
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation="softmax"))
model.summary()
model.compile(optimizer='adam',metrics=['accuracy'], loss="categorical_crossentropy")

In [ ]:
#Fitting X_train and y_train to the compiled model above 
history=model.fit(X_train,y_train,epochs=20,batch_size=32, validation_data=(X_test,y_test))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#Model evaluation
model.evaluate(X_test,y_test)

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
pred = model.predict(X_test)
pred = np.argmax(pred,axis = 1) 
y_true = np.argmax(y_test,axis = 1)

In [ ]:
#plotting confusion matrix
CM = confusion_matrix(y_true, pred)
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(5, 5))
plt.show()

In [ ]:
#Generating accuracy score and classication report
from sklearn.metrics import classification_report, accuracy_score
print("accuracy score :",accuracy_score(pred,y_true))
print(classification_report(pred,y_true))